In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

from tqdm.notebook import tqdm
import glob
import numpy as np
from utils_analysis import *
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from scipy.io import loadmat 

import seaborn as sns
import cv2
from skimage import data

In [2]:
training_methods = ["dino", "supervised"] 
depth_list = [8, 12]
num_models = 6
num_heads = 6

patch_size = 16
batch_size = 32

transform = pth_transforms.Compose([
    pth_transforms.ToTensor(),
    pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [3]:
image_labels = ["../dataset/popout/colors.png",
                "../dataset/popout/size.png",
                "../dataset/popout/alphabets.png",
                "../dataset/popout/orientation.png",
                "../dataset/popout/hatching.png"]
"""
image_labels = ["../dataset/popout/aaa.png",
                "../dataset/popout/bbb.png"]
"""

# load frame
imgs = []
# imagenet
for path in image_labels:#imagenet_path:
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if img.shape != (256, 256):
        img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
    imgs.append(img)    
#imgs.append(np.array(get_dot_box(bkg="ellipse")))

images = torch.stack([transform(img) for img in imgs])
num_imgs = len(images)

In [4]:
d_label = np.array([[7,8],[9,10]]) - 1

In [5]:
res_dict = {}
tm = training_methods[0]
for i, depth in enumerate(depth_list):
    print(depth)
    attn_depth = []
    for trial_idx in tqdm(range(num_models)):
        model, device = model_load(tm, trial_idx+1, depth, patch_size)
        #num_heads = model.num_heads
        with torch.inference_mode():
            attentions_list = model.get_fulllayers_selfattention(images.to(device))
        
        num_samples = len(images)
        attn_layer = []
        for j in range(2): 
            attentions = attentions_list[d_label[i,j]]
            attentions = attentions[:, :, 0, 1:].reshape(num_samples, num_heads, -1)
            attentions = attentions.detach().cpu().numpy()
            attn_layer.append(attentions[None, :, : ,:])
        attn_layer = np.vstack(attn_layer)
        attn_depth.append(attn_layer[None, :, :, :, :])
    attn_depth = np.vstack(attn_depth)
    attn_depth = attn_depth.transpose(1, 0, 3, 2, 4)#.reshape(2, -1, num_imgs, num_tokens)
    res_dict[str(depth)] = attn_depth 

8


  0%|          | 0/6 [00:00<?, ?it/s]

12


  0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
save_dir = "../results/"
np.savez_compressed(f"{save_dir}/dino_vit_attention_groups_toy.npz", **res_dict)